In [1]:
#generation debugging 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import networkx as nx
import rdkit
import math, random, sys
import numpy as np
import argparse
import os
from tqdm.auto import tqdm
import pickle
from hgraph import *
from hgraph.inc_graph import *
from hgraph.encoder import *
import matplotlib.pyplot as plt

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocab = '/home/marcase/hgraph2graph/data/chembl/cyclic_vocab_new.txt'
vocab = [x.strip("\r\n ").split() for x in open(vocab)]
vocab = PairVocab(vocab)
vocab.vocab[21][0]
vocab.hvocab

['CC',
 'CC(C)C',
 'CC(C)O',
 'CCC(=O)O',
 'CCC(C)C',
 'CCC(N)=O',
 'CCC1=CC=C(O)C=C1',
 'CCC1=CC=CC=C1',
 'CCC1=CN=CN1',
 'CCC1=CNC2=C1C=CC=C2',
 'CCCC(=O)O',
 'CCCC(N)=O',
 'CCCCCN',
 'CCCCNC(=N)N',
 'CCCSC',
 'CCO',
 'CCSCC(=O)CSCC',
 'CN1CCCC1C(=O)O',
 'CN1CCCC1C=O',
 'CNCC(=O)O',
 'CNCC=O',
 'NCC=O',
 'O=CC1CCCN1']

In [3]:
class Args:
    train = 'cyclic_preprocessed_new/'
    vocab = vocab
    save_dir = 'test/'
    atom_vocab = common_atom_vocab
    load_model = None
    seed = 7
    rnn_type = 'LSTM'
    hidden_size=125
    embed_size=250
    batch_size=1
    latent_size=32
    depthT=15
    depthG=15
    diterT=1
    diterG=3
    dropout=0.0
    lr = 1e-3
    clip_norm=5.0
    step_beta=0.001
    max_beta=1.0
    warmup=10000
    kl_anneal_iter=2000
    epoch=20
    anneal_rate=0.9
    anneal_iter=25000
    print_iter=50
    save_iter=5000
    model = '/home/marcase/hgraph2graph/ckpt/cyclic_new2/model.ckpt.140000'
    load_model = True
    nsample = 1
    max_nodes=200
    max_edges=400
    max_AA = 6
    max_sub_nodes = 50
    
args=Args()

model = HierVAE(args).cuda()

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [4]:
print("Model #Params: %dK" % (sum([x.nelement() for x in model.parameters()]) / 1000,))

Model #Params: 1353K


In [5]:
#from generate
model.load_state_dict(torch.load(args.model)[0])
model.eval()
torch.manual_seed(args.seed)
random.seed(args.seed)

In [7]:
with torch.no_grad():
    for _ in tqdm(range(args.nsample // args.batch_size)):
        smiles_list = model.sample(args.batch_size, greedy=True,max_nodes=200,max_edges=400,max_AA=6)

  0%|          | 0/1 [00:00<?, ?it/s]

(200, 2)
(200, 38)
Iteration: 0
C.CNCC=O
Most likely vocab elements to add: ['CNCC=O', 'NCC=O', 'CCO', 'CCCCCN', 'CN1CCCC1C=O']
Amino acid #: 0
Chosen vocab: CNCC=O
Atom index of new attachment: [3]
Iteration: 1
C.CNC(C=O)NCC=O
Most likely vocab elements to add: ['CNCC=O', 'NCC=O', 'CCO', 'CN1CCCC1C=O', 'CCC1=CN=CN1']
Amino acid #: 1
Chosen vocab: CNCC=O
Atom index of new attachment: [8]
Iteration: 2
C.CNC(C=O)NC(C=O)NCC=O
Most likely vocab elements to add: ['CNCC=O', 'CCO', 'CCCCNC(=N)N', 'CCCCCN', 'NCC=O']
Amino acid #: 2
Chosen vocab: CNCC=O
Atom index of new attachment: [12]
Iteration: 3
C.CNC(C=O)NC(C=O)NC(C=O)NCC=O
Most likely vocab elements to add: ['CNCC=O', 'CN1CCCC1C=O', 'CCO', 'NCC=O', 'CCC1=CC=C(O)C=C1']
Amino acid #: 3
Chosen vocab: CNCC=O
Atom index of new attachment: [16]
Iteration: 4
C.CNC(C=O)NC(C=O)NC(C=O)NC(C=O)NCC=O
Most likely vocab elements to add: ['CN1CCCC1C=O', 'CNCC=O', 'CCO', 'NCC=O', 'CCC1=CC=C(O)C=C1']
Amino acid #: 4
Chosen vocab: CN1CCCC1C=O
Atom index of

  0%|          | 0/1 [00:00<?, ?it/s]

Iteration: 18
C.CCSC(CCC(N)=O)C(=O)CSCC1CCN(C(C=O)NC(C=O)NC(C=O)NC(C=O)NC(C=O)NC)C1(C=O)NC(NCC=O)C(=O)C(O)CO
Most likely vocab elements to add: ['CCO', 'CCCCCN', 'CCSCC(=O)CSCC', 'CCC1=CNC2=C1C=CC=C2', 'CCCC(N)=O']
Chosen vocab: CCO
Atom index of new attachment: [19]
Iteration: 19
C.CCSC(CCC(N)=O)C(=O)CSCC1CCN(C(NC(C=O)NC(C=O)NC(C=O)NC(C=O)NC)C(=O)CO)C1(C=O)NC(NCC=O)C(=O)C(O)CO
Iteration: 20
C.CCSC(CCC(N)=O)C(=O)CSCC1CCN(C(NC(C=O)NC(C=O)NC(C=O)NC(C=O)NC)C(=O)CO)C1(C=O)NC(NCC=O)C(=O)C(O)CO
Iteration: 21
C.CCSC(CCC(N)=O)C(=O)CSCC1CCN(C(NC(C=O)NC(C=O)NC(C=O)NC(C=O)NC)C(=O)CO)C1(C=O)NC(NCC=O)C(=O)C(O)CO
Iteration: 22
C.CCSC(CCC(N)=O)C(=O)CSCC1CCN(C(NC(C=O)NC(C=O)NC(C=O)NC(C=O)NC)C(=O)CO)C1(C=O)NC(NCC=O)C(=O)C(O)CO
Most likely vocab elements to add: ['CCCCNC(=N)N', 'CCCCCN', 'CCC1=CNC2=C1C=CC=C2', 'CCSCC(=O)CSCC', 'CCCC(N)=O']
Chosen vocab: CCCCNC(=N)N
Atom index of new attachment: [11]
Iteration: 23
C.CCSC(CCC(N)=O)C(=O)CSCC1CCN(C(NC(C=O)NC(NC(C=O)NC(C=O)NC)C(=O)CCCNC(=N)N)C(=O)CO)C1(C=O)N

IndexError: list index out of range

In [13]:
args.vocab.is_backbone_hvocab('CN1CCCC1C=O')

True

In [ ]:
epoch = range(args.epoch)[0]
dataset = DataFolder(args.train,args.batch_size)
for param in model.parameters():
    # print(param)
    1

In [7]:
# #begin training the model
# for batch in tqdm(dataset):
#     total_step += 1
#     model.zero_grad()
#     loss, kl_div, wacc, iacc, tacc, sacc = model(*batch, beta=beta)

In [8]:
data_file0 = os.path.join(dataset.data_folder,'tensors-0.pkl')
with open(data_file0,'rb') as f:
    batches = pickle.load(f)


In [9]:
model.eval()
model(*batches[0],beta=beta)

(tensor(1.9945, device='cuda:0', grad_fn=<AddBackward0>),
 42.87664031982422,
 tensor(0.9921, device='cuda:0'),
 tensor(1., device='cuda:0'),
 tensor(1., device='cuda:0'),
 tensor(1., device='cuda:0'))

In [10]:
# nx.draw(batches[0][0][1])
# plt.show()

#just realized I have no idea what format the tensorized molecules are in. let's go back and check the tensorize function

In [11]:
#from model.sample()
batch_size = args.batch_size
latent_size = args.latent_size
root_vecs = torch.randn(batch_size, latent_size).cuda()

In [12]:
#from HierMPNDecoder.decode()
src_root_vecs, src_tree_vecs, src_graph_vecs = root_vecs,root_vecs,root_vecs
batch_size = len(src_root_vecs)
batch_size

1

In [13]:
tree_batch = IncTree(batch_size, node_fdim=2, edge_fdim=3)
# nx.draw(tree_batch.graph)

(100, 2)


In [14]:
hmpn = IncHierMPNEncoder(args.vocab, args.atom_vocab, args.rnn_type, args.embed_size, args.hidden_size, args.depthT, args.depthG, args.dropout)
test = hmpn.tree_encoder
test.depth

15

In [15]:
graph_batch = IncGraph(args.atom_vocab, batch_size, node_fdim=hmpn.atom_size, edge_fdim=hmpn.atom_size + hmpn.bond_size)
# nx.draw(graph_batch.graph)

(100, 38)


In [16]:
def get_cls_score(src_tree_vecs, batch_idx, cls_vecs, cls_labs):
    use_attention = False
    if use_attention:
        cls_cxt = self.attention(src_tree_vecs, batch_idx, cls_vecs, self.A_cls)
    else:
        cls_cxt = src_tree_vecs.index_select(index=batch_idx, dim=0).cuda()

    cls_vecs = torch.cat([cls_vecs, cls_cxt], dim=-1).cuda()
    clsNN = nn.Sequential(
                nn.Linear(args.hidden_size + args.latent_size, args.hidden_size),
                nn.ReLU(),
                nn.Dropout(args.dropout),
                nn.Linear(args.hidden_size, args.vocab.size()[0])
        ).cuda()
    iclsNN = nn.Sequential(
                nn.Linear(args.hidden_size + args.latent_size, args.hidden_size),
                nn.ReLU(),
                nn.Dropout(args.dropout),
                nn.Linear(args.hidden_size, args.vocab.size()[1])
        ).cuda()
    cls_scores = clsNN(cls_vecs)

    if cls_labs is None: #inference mode
        icls_scores = iclsNN(cls_vecs) #no masking
    else:
        vocab_masks = args.vocab.get_mask(cls_labs)
        icls_scores = iclsNN(cls_vecs) + vocab_masks #apply mask by log(x + mask): mask=0 or -INF
    return cls_scores, icls_scores

In [17]:
stack = [[] for i in range(batch_size)]
W_root = nn.Linear(args.latent_size, args.hidden_size).cuda()
init_vecs = src_root_vecs if args.latent_size == args.hidden_size else W_root(src_root_vecs)
itensor = torch.LongTensor([]).cuda()
batch_idx = itensor.new_tensor(range(batch_size))
cls_scores, icls_scores = get_cls_score(src_tree_vecs, batch_idx, init_vecs, None)
root_cls = cls_scores.max(dim=-1)[1]
icls_scores = icls_scores + args.vocab.get_mask(root_cls)
root_cls, root_icls = root_cls.tolist(), icls_scores.max(dim=-1)[1].tolist()

In [18]:
super_root = tree_batch.add_node() 
for bid in range(batch_size):
    clab, ilab = root_cls[bid], root_icls[bid]
    root_idx = tree_batch.add_node( batch_idx.new_tensor([clab, ilab]) )
    tree_batch.add_edge(super_root, root_idx) 
    stack[bid].append(root_idx)

    root_smiles = args.vocab.get_ismiles(ilab)
    new_atoms, new_bonds, attached = graph_batch.add_mol(bid, root_smiles, [], 0)
    tree_batch.register_cgraph(root_idx, new_atoms, new_bonds, attached)

In [19]:
# nx.draw(tree_batch.graph)

In [20]:
class HTuple():
    def __init__(self, node=None, mess=None, vmask=None, emask=None):
        self.node, self.mess = node, mess
        self.vmask, self.emask = vmask, emask
#invariance: tree_tensors is equal to inter_tensors (but inter_tensor's init_vec is 0)
tree_tensors = tree_batch.get_tensors()
graph_tensors = graph_batch.get_tensors()

htree = HTuple( mess = hmpn.tree_encoder.rnn.get_init_state(tree_tensors[1]) )
hinter = HTuple( mess = hmpn.tree_encoder.rnn.get_init_state(tree_tensors[1]) )
hgraph = HTuple( mess = hmpn.tree_encoder.rnn.get_init_state(graph_tensors[1]) )
h = hmpn.tree_encoder.rnn.get_hidden_state(htree.mess)
h[1 : batch_size + 1] = init_vecs #wiring root (only for tree, not inter)

In [21]:
#decode one step at a time
t=0 #up to max_decode_step, which is 100 by default
batch_list = [ bid for bid in range(batch_size) if len(stack[bid]) > 0 ]

In [22]:
batch_idx = batch_idx.new_tensor(batch_list)
cur_tree_nodes = [stack[bid][-1] for bid in batch_list]
subtree = batch_idx.new_tensor(cur_tree_nodes).cuda(), batch_idx.new_tensor([]).cuda()
subgraph = batch_idx.new_tensor( tree_batch.get_cluster_nodes(cur_tree_nodes) ).cuda(), batch_idx.new_tensor( tree_batch.get_cluster_edges(cur_tree_nodes) ).cuda()

In [24]:
htree, hinter, hgraph = hmpn(tree_tensors, tree_tensors, graph_tensors, htree, hinter, hgraph, subtree, subgraph)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)

In [ ]:
# sub_graph_tensors = hmpn.get_sub_tensor(graph_tensors,subgraph)[:-1]
# fnode, fmess, agraph, bgraph = sub_graph_tensors 
# subnode, submess = subgraph
# # if len(submess) > 0: 
# #     h = hmpn.graph_encoder.rnn.sparse_forward(hgraph.mess, fmess, submess, bgraph)
# atom_size = common_atom_vocab.size()
# bond_size = len(MolGraph.BOND_LIST) + MolGraph.MAX_POS
# rnn = LSTM(atom_size + bond_size, args.hidden_size, args.depthG)
# bgraph
# # h = rnn(fmess, bgraph)
# h = torch.zeros(fmess.size(0), args.hidden_size, device=fmess.device)
# c = torch.zeros(fmess.size(0), args.hidden_size, device=fmess.device)
# mask = torch.ones(h.size(0), 1, device=h.device)
# mask[0, 0] = 0 #first message is padding

# for i in range(args.depthG):
#     h_nei = index_select_ND(h, 0, bgraph)
#     c_nei = index_select_ND(c, 0, bgraph)
#     h,c = LSTM(fmess, h_nei, c_nei)
#     h = h * mask
#     c = c * mask
# # h = rnn.get_hidden_state(h)
# # h,c = h
# # mask = h.new_ones(h.size(0)).scatter_(0, submess, 0)
# # h = h * mask.unsqueeze(1)
# # c = c * mask.unsqueeze(1)

In [24]:
for i in range(args.depthG):
    h_nei = index_select_ND(h, 0, bgraph)
    c_nei = index_select_ND(c, 0, bgraph)

In [ ]:
def get_topo_score(self, src_tree_vecs, batch_idx, topo_vecs):
    use_attention = False
    if use_attention:
        topo_cxt = self.attention(src_tree_vecs, batch_idx, topo_vecs, self.A_topo)
    else:
        topo_cxt = src_tree_vecs.index_select(index=batch_idx, dim=0)
    return self.topoNN( torch.cat([topo_vecs, topo_cxt], dim=-1) ).squeeze(-1)
topo_scores = get_topo_score(src_tree_vecs, batch_idx, htree.node.index_select(0, subtree[0]))
topo_scores = torch.sigmoid(topo_scores)
if greedy:
    topo_preds = topo_scores.tolist()
else:
    topo_preds = torch.bernoulli(topo_scores).tolist()